<a href="https://colab.research.google.com/github/KaishvaShah/StyleIt/blob/main/SimpleRNN_inKeras_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
imdb, info = tfds.load("imdb_reviews",
                      with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYASZD/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYASZD/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim=16
max_length = 120
trunc_type= 'post'
oov_tok="<OOV>"

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 "'s": 21,
 'on': 22,
 'you': 23,
 'not': 24,
 'are': 25,
 'his': 26,
 'he': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 'all': 31,
 'at': 32,
 'by': 33,
 'they': 34,
 'an': 35,
 'who': 36,
 'so': 37,
 'from': 38,
 'like': 39,
 'her': 40,
 "'t": 41,
 'or': 42,
 'just': 43,
 'there': 44,
 'about': 45,
 'out': 46,
 "'": 47,
 'has': 48,
 'if': 49,
 'some': 50,
 'what': 51,
 'good': 52,
 'more': 53,
 'very': 54,
 'when': 55,
 'she': 56,
 'up': 57,
 'can': 58,
 'b': 59,
 'time': 60,
 'no': 61,
 'even': 62,
 'my': 63,
 'would': 64,
 'which': 65,
 'story': 66,
 'only': 67,
 'really': 68,
 'see': 69,
 'their': 70,
 'had': 71,
 'were': 72,
 'me': 73,
 'well': 74,
 'we': 75,
 'than': 76,
 'much': 77,
 'been': 78,
 'get': 79,
 'bad': 80,
 'will': 81,
 'people': 82,
 'do': 83,

In [8]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, 
                       truncating = trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                             input_length=max_length),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1568      
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 161,909
Trainable params: 161,909
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
num_epochs=30
history=model.fit(padded, training_labels_final, epochs=num_epochs, validation_data = (testing_padded, testing_labels_final))

Epoch 1/30
782/782 [==============================] - 26s 32ms/step - loss: 0.6936 - accuracy: 0.5076 - val_loss: 0.6894 - val_accuracy: 0.5379
Epoch 2/30
782/782 [==============================] - 25s 31ms/step - loss: 0.6140 - accuracy: 0.6644 - val_loss: 0.4660 - val_accuracy: 0.7905
Epoch 3/30
782/782 [==============================] - 25s 32ms/step - loss: 0.4192 - accuracy: 0.8169 - val_loss: 0.3971 - val_accuracy: 0.8184
Epoch 4/30
782/782 [==============================] - 24s 31ms/step - loss: 0.3303 - accuracy: 0.8678 - val_loss: 0.3816 - val_accuracy: 0.8340
Epoch 5/30
782/782 [==============================] - 25s 31ms/step - loss: 0.2922 - accuracy: 0.8859 - val_loss: 0.4355 - val_accuracy: 0.8204
Epoch 6/30
782/782 [==============================] - 24s 31ms/step - loss: 0.2516 - accuracy: 0.9042 - val_loss: 0.4725 - val_accuracy: 0.8059
Epoch 7/30
782/782 [==============================] - 24s 31ms/step - loss: 0.2060 - accuracy: 0.9228 - val_loss: 0.5112 - val_accuracy:

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                             input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9600      
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 170,261
Trainable params: 170,261
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss="binary_crossentropy",
             optimizer='adam',
             metrics=['accuracy'])
history=model.fit(padded, training_labels_final, epochs=num_epochs,
            validation_data = (testing_padded,testing_labels_final))

Epoch 1/30
782/782 [==============================] - 112s 93ms/step - loss: 0.3243 - accuracy: 0.8662 - val_loss: 0.3470 - val_accuracy: 0.8473
Epoch 2/30
782/782 [==============================] - 73s 94ms/step - loss: 0.2484 - accuracy: 0.9040 - val_loss: 0.3760 - val_accuracy: 0.8399
Epoch 3/30
782/782 [==============================] - 73s 93ms/step - loss: 0.1867 - accuracy: 0.9304 - val_loss: 0.4275 - val_accuracy: 0.8290
Epoch 4/30
782/782 [==============================] - 73s 93ms/step - loss: 0.1399 - accuracy: 0.9504 - val_loss: 0.5014 - val_accuracy: 0.8266
Epoch 5/30
782/782 [==============================] - 73s 93ms/step - loss: 0.1011 - accuracy: 0.9641 - val_loss: 0.5956 - val_accuracy: 0.8169
Epoch 6/30
782/782 [==============================] - 73s 93ms/step - loss: 0.0654 - accuracy: 0.9788 - val_loss: 0.6753 - val_accuracy: 0.8228
Epoch 7/30
782/782 [==============================] - 73s 94ms/step - loss: 0.0454 - accuracy: 0.9843 - val_loss: 0.8740 - val_accuracy

KeyboardInterrupt: ignored

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               12544     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 173,205
Trainable params: 173,205
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs=5
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data = (testing_padded, testing_labels_final))

Epoch 1/5
782/782 [==============================] - 74s 91ms/step - loss: 0.5144 - accuracy: 0.7258 - val_loss: 0.3610 - val_accuracy: 0.8433
Epoch 2/5
782/782 [==============================] - 70s 89ms/step - loss: 0.3092 - accuracy: 0.8762 - val_loss: 0.3516 - val_accuracy: 0.8470
Epoch 3/5
782/782 [==============================] - 70s 89ms/step - loss: 0.2437 - accuracy: 0.9088 - val_loss: 0.4054 - val_accuracy: 0.8357
Epoch 4/5
782/782 [==============================] - 70s 89ms/step - loss: 0.2030 - accuracy: 0.9253 - val_loss: 0.4571 - val_accuracy: 0.8314
Epoch 5/5
782/782 [==============================] - 70s 89ms/step - loss: 0.1702 - accuracy: 0.9416 - val_loss: 0.4189 - val_accuracy: 0.8227
